In [2]:
from graphviz import Digraph

class Automaton:
    def __init__(self, states, alphabet, start_state, accept_states, transitions):
        self.states = states
        self.alphabet = alphabet
        self.start_state = start_state
        self.accept_states = accept_states
        self.transitions = transitions

def nfa_to_dfa(nfa):
    dfa_transitions = {}
    dfa_states = [frozenset([nfa.start_state])]
    dfa_accept_states = set()
    unmarked_states = [frozenset([nfa.start_state])]
    
    while unmarked_states:
        current_state = unmarked_states.pop(0)
        if current_state in dfa_transitions:
            continue
        
        dfa_transitions[current_state] = {}
        
        for symbol in nfa.alphabet:
            new_state = frozenset(
                [nxt_state for state in current_state 
                            if state in nfa.transitions and symbol in nfa.transitions[state] 
                            for nxt_state in nfa.transitions[state][symbol]]
            )
            if new_state:
                dfa_transitions[current_state][symbol] = new_state
                if new_state not in dfa_states:
                    dfa_states.append(new_state)
                    unmarked_states.append(new_state)
        
        if any(state in nfa.accept_states for state in current_state):
            dfa_accept_states.add(current_state)

    return Automaton(dfa_states, nfa.alphabet, frozenset([nfa.start_state]), dfa_accept_states, dfa_transitions)

def visualize_automaton(automaton, filename):
    dot = Digraph()
    
    for state in automaton.states:
        label = "".join(state)
        if state in automaton.accept_states:
            dot.node(label, label, shape='doublecircle')
        else:
            dot.node(label, label)
    
    for state, transitions in automaton.transitions.items():
        for symbol, nxt_state in transitions.items():
            dot.edge("".join(state), "".join(nxt_state), label=symbol)
    
    dot.render(filename, format='png')

# Example input for an NFA
nfa = Automaton(
    states={'q0', 'q1', 'q2'},
    alphabet={'0', '1'},
    start_state='q0',
    accept_states={'q2'},
    transitions={
        'q0': {'0': ['q0'], '1': ['q0', 'q1']},
        'q1': {'1': ['q2']}
    }
)

# Convert to DFA
dfa = nfa_to_dfa(nfa)
visualize_automaton(dfa, "dfa_output")
